# Writing data to and reading data from a Database using Python

## Libraries and settings

In [7]:
# Libraries
import os
import sqlite3
import fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Function to close a sqlite db-connection
def check_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False

# Get current working directory
print(os.getcwd())

/workspaces/data_analytics/Week_02


## Create sqlite data base

In [8]:
# Create data base
conn = sqlite3.connect('supermarket_database.db') 
cursor = conn.cursor()

# Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'), '*.db')
for i in flist:
    print(i)

apartment_database.db
supermarket_database.db


## Create SQL-table in the database

In [9]:
cursor.execute('''CREATE TABLE IF NOT EXISTS supermarkets_table (
                    OrderId VARCHAR(50) PRIMARY KEY,
                    Brand VARCHAR(100),
                    Shop VARCHAR(50), 
                    City VARCHAR(100),
                    Street VARCHAR(200),
                    HouseNumber VARCHAR(20),
                    PostCode VARCHAR(10),
                    Latitude DECIMAL(10,8),
                    Longitude DECIMAL(11,8))''')

## Read data from file to data frame

In [10]:
# Read the prepared supermarket data
df = pd.read_csv('supermarkets_data_prepared.csv',
                  sep=',', 
                  encoding='utf-8')

# Select relevant columns and rename them for database
df_selected = df[['id', 'brand', 'shop', 'city', 'street', 
                  'housenumber', 'postcode', 'lat', 'lon']].copy()

# Rename columns to match database schema
df_selected.columns = ['OrderId', 'Brand', 'Shop', 'City', 'Street',
                       'HouseNumber', 'PostCode', 'Latitude', 'Longitude']

print(f"📈 Dataset loaded successfully!")
print(f"   Shape: {df_selected.shape}")
print(f"   Total supermarkets: {len(df_selected)}")
print(f"   Columns: {df_selected.shape[1]}")

print("\n🔍 First 5 records:")
df_selected.head()

📈 Dataset loaded successfully!
   Shape: (3392, 9)
   Total supermarkets: 3392
   Columns: 9

🔍 First 5 records:


,OrderId,Brand,Shop,City,Street,HouseNumber,PostCode,Latitude,Longitude
0,33126515,Spar,supermarket,NaN,NaN,NaN,NaN,47.155616,9.037915
1,36726161,Migros,supermarket,Uznach,Zürcherstrasse,25,8730.0,47.226191,8.980329
2,39768209,Coop,supermarket,Uznach,NaN,NaN,8730.0,47.225069,8.969981
3,39947904,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001.0,47.376732,8.542161
4,48932835,Migros,supermarket,Zürich,Wengistrasse,7,8004.0,47.375020,8.522895


## Write data to the SQL-table in data base

In [11]:
# Write DataFrame to SQLite table
df_selected.to_sql(name='supermarkets_table',
                   con=conn,
                   index=False,
                   if_exists='replace')

print("✅ Data successfully written to database!")
print(f"   {len(df_selected)} records inserted")

✅ Data successfully written to database!
   3392 records inserted


## Query the SQL-table

In [12]:
print("\n" + "="*60)
print("📍 TASK G: FILTER ALL SUPERMARKETS IN WINTERTHUR")
print("="*60)

# SQL query to find all supermarkets in Winterthur
query_winterthur = '''
    SELECT OrderId, Brand, Shop, Street, HouseNumber, PostCode
    FROM supermarkets_table
    WHERE City = 'Winterthur'
    ORDER BY Brand, Street
'''

cursor.execute(query_winterthur)
winterthur_results = cursor.fetchall()

# Convert to DataFrame for better display
df_winterthur = pd.DataFrame(winterthur_results, 
                            columns=['OrderId','Brand','Shop','Street',
                                    'HouseNumber','PostCode'])

print(f"\n🏪 Found {len(df_winterthur)} supermarkets in Winterthur")

if len(df_winterthur) > 0:
    print("\nAll supermarkets in Winterthur:")
    print("-" * 60)
    display(df_winterthur)
    
    # Statistics for Winterthur
    print(f"\n📊 Statistics for Winterthur:")
    print(f"   Total locations: {len(df_winterthur)}")
    
    # Count by brand
    cursor.execute('''
        SELECT Brand, COUNT(*) as Count
        FROM supermarkets_table
        WHERE City = 'Winterthur' AND Brand IS NOT NULL
        GROUP BY Brand
        ORDER BY Count DESC
    ''')
    
    brand_stats = cursor.fetchall()
    
    if brand_stats:
        print("\n   Supermarkets by brand:")
        for brand, count in brand_stats:
            print(f"     • {brand}: {count} location(s)")
    
    # Count by postcode
    cursor.execute('''
        SELECT PostCode, COUNT(*) as Count
        FROM supermarkets_table
        WHERE City = 'Winterthur' AND PostCode IS NOT NULL
        GROUP BY PostCode
        ORDER BY PostCode
    ''')
    
    postcode_stats = cursor.fetchall()
    
    if postcode_stats:
        print("\n   Distribution by postcode:")
        for postcode, count in postcode_stats:
            print(f"     • {postcode}: {count} supermarket(s)")
            
else:
    print("\n⚠️ No supermarkets found in Winterthur")
    print("\nChecking data for troubleshooting...")
    
    # Check what cities are available
    cursor.execute('''
        SELECT DISTINCT City
        FROM supermarkets_table
        WHERE City LIKE '%Winter%' OR City LIKE '%winter%'
        LIMIT 10
    ''')
    
    similar_cities = cursor.fetchall()
    
    if similar_cities:
        print("\n📍 Cities with similar names:")
        for city in similar_cities:
            print(f"   - {city[0]}")
    
    # Show top cities for reference
    cursor.execute('''
        SELECT City, COUNT(*) as Count
        FROM supermarkets_table
        WHERE City IS NOT NULL
        GROUP BY City
        ORDER BY Count DESC
        LIMIT 10
    ''')
    
    top_cities = cursor.fetchall()
    
    print("\n📊 Top 10 cities in database:")
    for city, count in top_cities:
        print(f"   - {city}: {count} supermarkets")


📍 TASK G: FILTER ALL SUPERMARKETS IN WINTERTHUR

🏪 Found 27 supermarkets in Winterthur

All supermarkets in Winterthur:
------------------------------------------------------------


,OrderId,Brand,Shop,Street,HouseNumber,PostCode
0,706203439,None,supermarket,Bankstrasse,8/12,8400.0
1,677194765,None,supermarket,Frauenfelderstrasse,69,8404.0
2,4978196565,None,supermarket,Grüzefeldstrasse,24,8400.0
3,4209642918,None,supermarket,In der Au,10,8406.0
4,4125136758,None,supermarket,Neuwiesenstrasse,6,8400.0
5,4095400178,None,supermarket,Römerstrasse,77,8404.0
6,8519878942,None,supermarket,Römerstrasse,167,8404.0
7,709022324,None,supermarket,Steinberggasse,18,8400.0
8,4109460421,None,supermarket,Wartstrasse,14,8400.0
9,10221262155,None,supermarket,Wülflingerstrasse,73/75,8400.0



📊 Statistics for Winterthur:
   Total locations: 27

   Supermarkets by brand:
     • Migros: 9 location(s)
     • Denner: 2 location(s)
     • ALDI: 2 location(s)
     • Migros-Outlet: 1 location(s)
     • Coop: 1 location(s)
     • Alnatura: 1 location(s)

   Distribution by postcode:
     • 8400.0: 12 supermarket(s)
     • 8404.0: 6 supermarket(s)
     • 8405.0: 2 supermarket(s)
     • 8406.0: 5 supermarket(s)
     • 8408.0: 1 supermarket(s)
     • 8409.0: 1 supermarket(s)


### Additional SQL-queries

In [13]:
print("=== ADDITIONAL SQL-QUERIES ===\n")

# Verbindung zur Datenbank öffnen (falls geschlossen)
conn = sqlite3.connect('apartment_database.db')
cursor = conn.cursor()

# c) Filter apartments with >= 4.0 rooms and >= 100m2
print("c) APARTMENTS WITH >= 4.0 ROOMS AND >= 100m²:")
print("-" * 50)

query_c = '''SELECT *
             FROM apartments_table
             WHERE Rooms >= 4.0 AND Area >= 100'''

cursor.execute(query_c)
results_c = cursor.fetchall()

# Convert to DataFrame for better display
df_c = pd.DataFrame(results_c, 
                   columns=['OrderId', 'Address', 'Rooms', 'Area', 'Price'])

print(f"Found {len(df_c)} apartments with >= 4.0 rooms and >= 100m²")
print("\nFirst 10 results:")
if len(df_c) > 0:
    print(df_c.head(10))
else:
    print("No apartments found matching criteria")

print("\n" + "="*60)

=== ADDITIONAL SQL-QUERIES ===

c) APARTMENTS WITH >= 4.0 ROOMS AND >= 100m²:
--------------------------------------------------
Found 239 apartments with >= 4.0 rooms and >= 100m²

First 10 results:
         OrderId                                         Address  Rooms  \
0  1693998205-26           Frowiesstrasse 36, 8344 Bäretswil, ZH    5.5   
1  1693998205-27          Kanzleistrasse 15, 8418 Schlatt ZH, ZH    4.5   
2  1693998205-29                   Bolletweg 14, 8934 Knonau, ZH    5.5   
3  1693998205-31               Adlergass 6, 8805 Richterswil, ZH    5.5   
4  1693998205-33  Haldenstrasse 67, 8602 Wangen b. Dübendorf, ZH    4.5   
5  1693998205-36        Albisriederstrasse 392a, 8047 Zürich, ZH    5.5   
6  1693998205-37        Tisliacherstrasse 8, 8320 Fehraltorf, ZH    5.5   
7  1693998205-38        Buckwiesstrasse 12, 8700 Küsnacht ZH, ZH    5.5   
8  1693998205-39     Binzigerstrasse 52, 8707 Uetikon am See, ZH    6.5   
9  1693998205-40               Hinterdorfstr. 4, 8

### SQL-query Additional Analysis

In [14]:
print("\n" + "="*60)
print("📊 ADDITIONAL ANALYSIS")
print("="*60)

# Overall statistics
cursor.execute('''
    SELECT 
        COUNT(DISTINCT Brand) as unique_brands,
        COUNT(DISTINCT City) as unique_cities,
        COUNT(*) as total_supermarkets
    FROM supermarkets_table
''')

stats = cursor.fetchone()

print("\n📈 Database Overview:")
print(f"   • Total supermarkets: {stats[2]}")
print(f"   • Unique brands: {stats[0]}")
print(f"   • Unique cities: {stats[1]}")

# Top 5 brands overall
print("\n🏆 Top 5 Supermarket Brands (Switzerland):")

cursor.execute('''
    SELECT Brand, COUNT(*) as Count
    FROM supermarkets_table
    WHERE Brand IS NOT NULL
    GROUP BY Brand
    ORDER BY Count DESC
    LIMIT 5
''')

top_brands = cursor.fetchall()

for i, (brand, count) in enumerate(top_brands, 1):
    print(f"   {i}. {brand}: {count} stores")


📊 ADDITIONAL ANALYSIS


OperationalError: no such table: supermarkets_table

### Writing supermarket data to and reading data from a Database using Python

In [15]:
# Libraries
import os
import sqlite3
import fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Function to close a sqlite db-connection
def check_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False

# Get current working directory
print(os.getcwd())

/workspaces/data_analytics/Week_02


In [16]:
# Create supermarket database
conn = sqlite3.connect('supermarket_database.db') 
cursor = conn.cursor()

# Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'), '*.db')
for i in flist:
    print(i)

apartment_database.db
supermarket_database.db


In [17]:
cursor.execute('''CREATE TABLE IF NOT EXISTS supermarkets_table (
                    OrderId VARCHAR(50),
                    Brand VARCHAR(100),
                    Shop VARCHAR(50), 
                    City VARCHAR(100),
                    Street VARCHAR(200),
                    HouseNumber VARCHAR(20),
                    PostCode VARCHAR(10),
                    Latitude DECIMAL(10,8),
                    Longitude DECIMAL(11,8))''')

# Confirm changes to the table
conn.commit()

In [ ]:
# Read supermarket data from file
df = pd.read_csv('supermarkets_data_prepared.csv',
                  sep=',', 
                  encoding='utf-8')[['id', 'brand', 'shop', 'city', 'street', 'housenumber', 'postcode', 'lat', 'lon']]

print(df.shape)
df.head(5)

In [18]:
# Write data to the SQL-table in database
df.to_sql(name = 'supermarkets_table',
          con = conn,
          index = False,
          if_exists = 'replace')

3392

In [19]:
# Query the SQL-table
cursor.execute('''SELECT *
               FROM supermarkets_table
               WHERE brand IS NOT NULL
               LIMIT 10''')

df_test = pd.DataFrame(cursor.fetchall(), 
                      columns=['OrderId','Brand','Shop','City','Street','HouseNumber','PostCode','Latitude','Longitude'])    
df_test

ValueError: 9 columns passed, passed data had 10 columns

## SQL-Query: Supermärkte in Winterthur

In [ ]:
# g) SQL-query to filter all supermarkets in Winterthur
print("=== SUPERMARKETS IN WINTERTHUR ===")

query_winterthur = '''SELECT *
                     FROM supermarkets_table
                     WHERE City = 'Winterthur' '''

cursor.execute(query_winterthur)
winterthur_results = cursor.fetchall()

df_winterthur = pd.DataFrame(winterthur_results, 
                           columns=['OrderId','Brand','Shop','City','Street','HouseNumber','PostCode','Latitude','Longitude'])

print(f"Found {len(df_winterthur)} supermarkets in Winterthur:")
print(df_winterthur)

if len(df_winterthur) == 0:
    print("\n⚠️ No supermarkets found in Winterthur.")
    print("Checking available cities...")
    
    # Show available cities
    cursor.execute('''SELECT DISTINCT City
                     FROM supermarkets_table 
                     WHERE City IS NOT NULL
                     ORDER BY City
                     LIMIT 20''')
    
    cities = cursor.fetchall()
    print("Available cities in database:")
    for city in cities:
        if city[0]:  # Skip None values
            print(f"  - {city[0]}")

In [ ]:
# Close db connection (if open)
try:
    if check_conn(conn):
        conn.close()
    else:
        pass
except:
    pass

# Status (True = open, False = closed)
print(check_conn(conn))

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

## Close db connection (if open)

In [ ]:
# Close db connection (if open)
try:
    if check_conn(conn):
        conn.close()
    else:
        pass
except:
    pass

# Status (True = open, False = closed)
print(check_conn(conn))

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')